In [1]:
import pandas as pd

In [2]:
df= pd.read_csv('Restaurant_Reviews.tsv', sep='\t')

In [3]:
df.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [4]:
msg='Wow... Loved this place.'

In [5]:
#Lower case
msg=msg.lower()
msg

'wow... loved this place.'

In [6]:
# remove unwanted characters
import re

In [7]:
msg1=re.sub('[^a-z]', ' ', msg)

In [8]:
!pip install nltk

In [9]:
import nltk

In [10]:
#download stopwords
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [11]:
nltk.corpus.stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [12]:
list_of_words= msg1.split()

In [13]:
list_of_words

['wow', 'loved', 'this', 'place']

In [14]:
list_without_stopwords=[]
for word in list_of_words:
    if word in nltk.corpus.stopwords.words('english'):
        print(word, 'is stopword')
    else:
        list_without_stopwords.append(word)

this is stopword


In [15]:
list_without_stopwords

['wow', 'loved', 'place']

In [16]:
#Using list comprehension
l_wo_s= [word for word in list_of_words if word not in nltk.corpus.stopwords.words('english')]

In [17]:
l_wo_s

['wow', 'loved', 'place']

In [18]:
#Stemming (base word)
from nltk.stem import PorterStemmer

In [19]:
ps= PorterStemmer()

In [20]:
list_of_stemmed_words= [ps.stem(word) for word in l_wo_s]
list_of_stemmed_words

['wow', 'love', 'place']

In [21]:
msg=' '.join(list_of_stemmed_words)
msg

'wow love place'

In [22]:
#Using generalised Function
def get_processed(msg):
    msg= re.sub('[^a-zA-Z]',' ', msg)
    msg=msg.lower()
    list_A= msg.split()
    list_B= [ps.stem(word) for word in list_A if word not in nltk.corpus.stopwords.words('english')]
    msg=' '.join(list_B)
    return msg

In [23]:
df['Review'].apply(get_processed)

0                                         wow love place
1                                             crust good
2                                     tasti textur nasti
3      stop late may bank holiday rick steve recommen...
4                                select menu great price
                             ...                        
995                        think food flavor textur lack
996                               appetit instantli gone
997                         overal impress would go back
998    whole experi underwhelm think go ninja sushi n...
999    wast enough life pour salt wound draw time too...
Name: Review, Length: 1000, dtype: object

In [24]:
# Build BoW model
from sklearn.feature_extraction.text import CountVectorizer

In [25]:
BoW=CountVectorizer()

In [26]:
BoW_table= BoW.fit_transform(df['Review'].apply(get_processed))
BoW_table

<1000x1565 sparse matrix of type '<class 'numpy.int64'>'
	with 5372 stored elements in Compressed Sparse Row format>

In [27]:
X= BoW_table.toarray()

In [28]:
X.shape

(1000, 1565)

In [29]:
sum(X[0])

3

In [30]:
y= df['Liked']

In [31]:
#Build ML Model
from sklearn.ensemble import RandomForestClassifier

In [32]:
model= RandomForestClassifier(random_state=56)

In [33]:
model.fit(X,y)

RandomForestClassifier(random_state=56)

In [34]:
model.score(X,y)

0.997

In [37]:
def get_sentiment(review):
    review= get_processed(review)
    #model.predict(review)
    a1= BoW.transform([review])
    a1= a1.toarray()
    return model.predict(a1)[0]

In [38]:
get_sentiment('liked the place')

0

In [39]:
get_sentiment('Very near to Swargate and good food')

1

In [40]:
get_sentiment('Food was not good')

1